### Coleta usando a API do twitter

In [1]:
import oauth2
import sys
import os
import json
import time
import csv
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
import threading
import re

In [2]:
def criaJsonSaida(json_input):
    return {
        "id": json_input["id"],
        "data_criacao": json_input["created_at"],
        "text": json_input["full_text"].lower(),
        "usuario": json_input["user"]["name"],
        "e_sobre_a_empresa": None
    }

def get_requisicao(cliente, max_id, termo):
    requisicao = cliente.request(
        'https://api.twitter.com/1.1/search/tweets.json?q={termo}&tweet_mode=extended&result_type=recent&lang=pt-br&count=3&{max_id}'.format(max_id=max_id, termo=termo))
    decodificar = requisicao[1].decode()
    try:
        objeto = json.loads(decodificar)
        return objeto['statuses']
    except Exception:
        print(objeto)

def coleta(api_key, api_secret_key, token_key, token_secret_key, termo, num_tweets, arquivo_saida):
    consumer = oauth2.Consumer(api_key, api_secret_key)
    token = oauth2.Token(token_key, token_secret_key)
    cliente = oauth2.Client(consumer, token)
    strUrl = ""
    max_id = ""
    num_tweets_collected = 0

    tweetsColetados = {
        "tweets": []
    }

    dataset = {}
    
    try:
        # coleta os tweets
        while(num_tweets_collected < num_tweets):
            tweets = get_requisicao(cliente, max_id, termo)            
            for tweet in tweets:
                tweet_saida = criaJsonSaida(tweet)
                par_unico = (tweet_saida["usuario"], tweet_saida["data_criacao"])
                if dataset.get(par_unico) == None:
                    dataset[par_unico] = { "id" : tweet_saida["id"]}
                    tweetsColetados["tweets"] += [criaJsonSaida(tweet)]
                
                max_id = "max_id={max_id}&".format(max_id=tweet['id'])
                num_tweets_collected += 1
    except Exception as e:
        print(e)
    finally:
        with open(arquivo_saida, 'w') as f:
            f.write(json.dumps(tweetsColetados["tweets"], indent=2))


In [4]:
# inicializa keys
api_key = '7lAndD8Xck9bXWN8XIma6poIR'
api_secret_key = '65RHfWySuVQzOEYxLRBIBYRWM92AdLOOjtBB1ZrrEaDzgGR55p'
token_key = '1117583656887947264-Nve0qICFP5gL3YH6NyqyanT1HOd8S4'
token_secret_key = 'c2N8kmDIhYLLYoreWGrohRamGzf0U4MhbNNxRP8PUfAzP'
termo = 'sadia'
num_tweets = 300
arquivo_com_tweetes_coletados = 'tweets_coleta_iterativa.json'

try:
    coleta(api_key, api_secret_key, token_key,
           token_secret_key, termo, num_tweets, arquivo_com_tweetes_coletados)
except Exception as e:
    print(e)

{'errors': [{'code': 88, 'message': 'Rate limit exceeded'}]}
'NoneType' object is not iterable


### Gerando csv com os tweets a serem rotulados

In [5]:
with open(arquivo_com_tweetes_coletados, 'r') as f:
    dic_tweets = json.loads(f.read())
    tweetsDF = pd.read_json(json.dumps(dic_tweets))
nome_arquivo_com_tweetes_rotulados = 'tweets.csv'
tweetsDF.to_csv(nome_arquivo_com_tweetes_rotulados, quoting=csv.QUOTE_ALL)

### Lendo o csv com os tweets rotulados

In [6]:
tweets_rotuladosDF = pd.read_csv(nome_arquivo_com_tweetes_rotulados)

In [7]:
tweets_rotuladosDF

,Unnamed: 0,data_criacao,e_sobre_a_empresa,id,text,usuario
0,0,Sat Apr 20 02:16:46 +0000 2019,NaN,1119424638239608832,ainda bem também que tenho todas as ligações d...,ilariê
1,1,Sat Apr 20 02:03:54 +0000 2019,NaN,1119421403080609792,"@crf_gusta entro não po, conversa sadia, conve...",insta: @igo_r02
2,2,Sat Apr 20 01:57:29 +0000 2019,NaN,1119419787627696128,@viinicius_157 @ig_siilvaa conversa tá sadia r...,Barbosa
3,3,Sat Apr 20 01:54:49 +0000 2019,NaN,1119419116929077249,@mr_pi31415 @viniciusdeborba @brazbruno05 @rii...,mAThEUs Conradt
4,4,Sat Apr 20 01:46:12 +0000 2019,NaN,1119416946724880387,@edir_braga @rodrigodasilva a chance pra se te...,O Doutrinador ☭


### Fazendo um pré-processamento nos textos dos tweets

### Identificando os brigramas e trigramas encontrados nos twittes

In [8]:
tknzr = TweetTokenizer()

ngram = {}

for indice, linha in tweets_rotuladosDF.iterrows() :
    tokens = tknzr.tokenize(linha[tweets_rotuladosDF.text.name])
    for count in range(len(tokens)):
        if(tokens[count] == 'Sadia' or tokens[count] == 'sadia'):
            bigram = (tokens[count - 1], tokens[count])
            trigram = (tokens[count - 2], tokens[count - 1], tokens[count])
            if ngram.get(bigram) == None:
                ngram[bigram] = {"frequencia" : 1, "id_tweets" : [indice]}
            else:
                valor = ngram.get(bigram)
                valor["frequencia"] += 1
                valor["id_tweets"] += [indice]
                
            if ngram.get(trigram) == None:
                ngram[trigram] = {"frequencia" : 1, "id_tweets" : [indice]}
            else:
                valor = ngram.get(trigram)
                valor["frequencia"] += 1
                valor["id_tweets"] += [indice]
                

In [9]:
ngram = sorted(ngram.items(), key=lambda x: x[1]['frequencia'], reverse=True)

Imprimindo os n mais frequetes

In [10]:
n = 100
contador = 1
for tupla in ngram:
    if(contador > n):
        break
    chaves = tupla[0]
    valores = tupla[1]
    print("freq: " + str(valores['frequencia']) + " " + str(chaves))
    contador += 1

freq: 2 ('conversa', 'sadia')
freq: 1 ('conversa', 'tá', 'sadia')
freq: 1 ('@riikivargas', 'brincadeira', 'sadia')
freq: 1 (',', 'conversa', 'sadia')
freq: 1 ('uma', 'amizade', 'sadia')
freq: 1 ('brincadeira', 'sadia')
freq: 1 ('amizade', 'sadia')
freq: 1 ('uma', 'conversa', 'sadia')
freq: 1 ('tá', 'sadia')
